In [ ]:
# Pytohn script for Ford Credit/Audience Segmentation Cluster Analysis
# Alejandro Pineda, Data Scientist, VMLYR
# 3/08/2023

# Load Libraries (install as necessary)
import numpy as np
import pandas as pd
import sklearn
import os
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist 
#from sklearn import StandardScalar
from kmodes.kprototypes import KPrototypes
# seed # (might be possible to set numpy seed globally, but not advised? Investigate further)
rando_n = 1234
from lightgbm import LGBMClassifier
import shap
from sklearn.model_selection import cross_val_score
from plotly import graph_objects as go
from tqdm import tqdm
import re

from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn import datasets

In [ ]:
def data_loader(file_name, col_drops):
    """
    Loads data//drops columns where necessary//provides shape and dtype info
    
    """
    original_dat = pd.read_csv(file_name, low_memory=False)
    og_shape = original_dat.shape
    
    dat_dropped = original_dat.drop(columns=col_drops, errors = 'ignore') # see if we can do try/except here
    drop_shape = dat_dropped.shape
    
    dat_types = pd.DataFrame(dat_dropped.dtypes)
    print("Header for new data:")
    print(dat_dropped.head())
    print(f"Shape of original data: {og_shape}" )
    print(f"Shape of new data: {drop_shape}")

    
    return dat_dropped, drop_shape, dat_types

In [ ]:
# Some of our column names have weird names or tags 
# (legacy data will use tags to tell use what database it came from)
# use reg-ex to pattern match and clean

def pattern_find(columns, pattern=str):
    """
    Looks for columns that match a specific pattern
    """
    reg_pattern = ".*" + pattern
    display(f"Looking for the following pattern: {reg_pattern}")
    
    r = re.compile(reg_pattern)
    col_list = list(filter(r.match, columns)) 
    display(col_list[0:5])
    return col_list
    
pattern = ""
cols_list = list(dat.columns)

drop_cols = pattern_find(columns = cols_list, pattern = cluster_pat )

dat.drop(cluster_cols, axis=1, inplace=True)
dat.drop(release_cols, axis=1, inplace=True)

In [ ]:
# Handling missing data and identifying categorical columns by index

# important function, but it's long and tedious. 
# break this down into distinct tasks (3/8)

def prep_dat(data_frame, thresh=None, mean_fill = False, dat_fill=None, limit=None, na_drop=None):

    """
    Fills in and/or drops missing values. 
    
    dat_fill (str): ‘backfill’, ‘bfill’, ‘pad’, ‘ffill’, None (default)
    limit (int): the maximum number of consecutive NaN allowed
    na_drop (str): Do you want rows or columns dropped? 'columns','index', or None (default) 
    thresh (int): Do you want to just require a specific # of non-NA values?
    mean_fill: Do you want missing values in numerical columns filled in with column mean? Boolean
    """
    
    data_frame2 = data_frame
    
    num_cols = data_frame2.select_dtypes(exclude=['object']).columns
    data_frame2[num_cols] = data_frame2[num_cols].astype(np.float32)
    
    # handle numerical missing nulls with the mean
    if mean_fill:
        num_features = list(num_cols)
        data_frame2[num_features]  = data_frame2[num_features].fillna(data_frame2.mean())
    
    # handle categorical missing nulls with the specified method
    if dat_fill:
        data_frame2 = data_frame2.fillna(method=dat_fill, limit = limit)
    
    if na_drop:  
        data_frame2 = data_frame2.dropna(axis=na_drop, thresh=thresh)
    
    display(f"Data loaded for this model:")
    display(data_frame.head())
    
    display("Data after pre-processing: ")
    num_features = list(data_frame2.select_dtypes(exclude=['object']).columns)
    data_frame2[num_features] = StandardScaler().fit_transform(data_frame2[num_features])
    
    
    # Identifying categorical variables based on index
    all_cols = list(data_frame2.columns)
    
    cat_col_names = list(set(all_cols) - set(num_features))
    cat_index = list()
    for col in cat_col_names:
        # find the index no
        index_loc = data_frame2.columns.get_loc(col)
        cat_index.append(index_loc)

    display(f"n_categorical features: {len(cat_index)}")
    display(f"n_numerical features: {len(num_features)}")
    
    display(f"Shape of the original data set: {data_frame.shape} // Shape of new data: {data_frame2.shape}")
    display("Pre-processed data:")
    display(data_frame2.head())
    return data_frame2
        



In [ ]:
# Columns we know we won't need

lil_trash = []


file_name = ''
trash_cols = pd.read_csv(file_name, dtype = 'str')
trash_cols



col_names = trash_cols.columns
trash = list()

for col in col_names:
    col = trash_cols[f'{col}'].dropna()
    trash.append(col)


trash = [item for sublist in trash for item in sublist]
full_trash = trash + lil_trash




In [ ]:
filename = ''

dat = pd.DataFrame()
dat_shape = tuple()
dtype = pd.DataFrame()

dat, dat_shape, dtype = data_loader(file_name = filename, col_drops=full_trash)

In [ ]:
dtype

In [ ]:

# Checking for null values (data quality is important!)

nulls = dat.isnull().agg('sum')
# nulls.to_csv('', index = False)
nulls


In [ ]:
#dat.groupby('RACE').count()
var_count = ' '
dat.groupby(var_count).count()

In [ ]:
# Function to find integer variables
# wrote this as a one off, should be simpler. can be re-purposed for data manipulations based on dtype.

"""

def RepresentsInt(var):
    if dat[var].dtype == 'object':
        return False
    try: 
        if (dat[var].fillna(-9999) % 1 == 0).all(): # check the modulo operator to see if there's a decimal
            return True
        else:
            return False
    except ValueError:
        return False
    except TypeError:
        print(dat[var].dtype)
        
# map integers to int64 (size consideration?)
counter = 0

for var in dat.columns:
    if RepresentsInt(var) == True:
        print(var)
        dat[var] = dat[var].astype('int64')
        counter += 1

print(counter)

"""
dat['']  = dat[''].astype('float32')

In [ ]:
dtype_dict = dat.dtypes.to_dict()

In [ ]:
# number of unique groups in categorical var

dat.groupby().nunique()

In [ ]:
#### Adding a binary var for DMA codes from the Southeast 6/1
### commented out on 3/10

"""
dat.dropna(subset=['N2DMA'])
display("Current shape of df: " + str(dat.shape))

#dat['N2DMA'] = dat['N2DMA'].astype('int64')
se_codes_list = [525, 524, 520, 630, 575, 522, 606, 691, 557, 503, 698, 507]

#dat['SE_DMA'] = dat['N2DMA'].isin(se_codes_list)




se_dat = dat[dat['N2DMA'].isin(se_codes_list)]

non_se_dat = dat[dat['N2DMA'].isin(se_codes_list) == False]

display('Shape of se_dat: ' + str(se_dat.shape))
display('Shape of non_se_dat' + str(non_se_dat.shape))

"""





In [ ]:
#limit = non_se_dat.shape[0] * 0.7

nonse_prepped = prep_dat(non_se_dat, thresh=None, mean_fill = True, dat_fill='ffill', limit=25, na_drop='columns')

In [ ]:
se_prepped = prep_dat(se_dat, thresh=None, mean_fill = True, dat_fill='ffill', limit=25, na_drop='columns')

In [ ]:
def kproto_trainer(data_frame, k, random_state, num_init=3):
    
    proto_dat = data_frame
    
    num_features = list(proto_dat.select_dtypes(exclude=['object']).columns)
    all_cols = list(proto_dat.columns)
    
    cat_col_names = list(set(all_cols) - set(num_features))
    cat_index = list()
    for col in cat_col_names:
        # find the index no
        index_loc = proto_dat.columns.get_loc(col)
        cat_index.append(index_loc)
    
    
    #### Initialize and train model on give df ####    
    
    # Leaving init and verbose as settings we don't need to worry about for now
    kproto = KPrototypes(n_clusters=k, init='Huang', verbose=2, random_state=random_state, n_init = num_init)
    display(f"Training k-prototype model with {k} clusters and {num_init} centroid inits.")
    
    
    clusters = kproto.fit_predict(proto_dat, categorical=cat_index)
    proto_dat['label'] = kproto.predict(proto_dat, categorical = cat_index)
    data_frame_lab = proto_dat
    
    return kproto, clusters, data_frame_lab
    

In [ ]:
# Elbow plot with cost (will take a LONG time)
# Ran on 6/21 - running with 3 clusters


elbow_samp = nonse_prepped.sample(n=5000, replace=False, random_state=rando_n)


proto_dat = elbow_samp
    
num_features = list(proto_dat.select_dtypes(exclude=['object']).columns)
all_cols = list(proto_dat.columns)
    
cat_col_names = list(set(all_cols) - set(num_features))
cat_index = list()

for col in cat_col_names:
    # find the index no
    index_loc = proto_dat.columns.get_loc(col)
    cat_index.append(index_loc)
    
costs = []
n_clusters = []
clusters_assigned = []

for i in tqdm(range(2, 9)):
    try:
        kproto = KPrototypes(n_clusters= i, init='Cao', verbose=2)
        clusters = kproto.fit_predict(proto_dat, categorical = cat_index)
        costs.append(kproto.cost_)
        n_clusters.append(i)
        clusters_assigned.append(clusters)
    except:
        print(f"Can't cluster with {i} clusters")
        
fig = go.Figure(data=go.Scatter(x=n_clusters, y=costs ))
fig.show()









In [ ]:

nonse_prepped = nonse_prepped.sample(n=11000, replace=False, random_state=rando_n)
nonse_prepped.head()
nonse_prepped.shape


In [ ]:
se_prepped = se_prepped.sample(n=1100, replace=False, random_state=rando_n)
se_prepped.head()
se_prepped.shape

In [ ]:
# Training the model

kproto_model, kproto_cluster, lab_df = kproto_trainer(data_frame=nonse_prepped, random_state=rando_n, k=3, num_init=3)



In [ ]:
kproto_model

In [ ]:
import pickle

model_file = "kproto_cluster_6_22.pkl"  

#pickle.dump(kproto_model, open(model_file, 'wb'))



In [ ]:

with open(model_file, 'rb') as file:  
    pickpled_kproto = pickle.load(file)

pickpled_kproto




In [ ]:
# Print cluster centroids of the trained model.
print(pickpled_kproto.cluster_centroids_)

# Print training statistics
print(kproto_model.cost_)
print(kproto_model.n_iter_)


#kproto_model, kproto_cluster, lab_df
lab_df

In [ ]:
#color_dict = { 0:'red', 1:'blue', 2:'black', 3:'green', 4:'purple' }

#c = [color_dict[i] for i in lab_df['label'] ] 
#c

In [ ]:
from prince import FAMD



famd = FAMD(n_components=2, n_iter=3, check_input=True, engine='auto', random_state=rando_n)

famd = famd.fit(lab_df.drop('label', axis='columns'))
famd.fit(lab_df)
famd.transform(lab_df)

clx = famd.plot_row_coordinates(lab_df, figsize=(15,10), color_labels = lab_df['label'])

clx.get_figure().savefig('clusters_622.png')

In [ ]:
print(pd.Series(kproto_cluster).value_counts())

In [ ]:
# Grabbing F1 score to see how well groupings based on categories predicts data
# https://en.wikipedia.org/wiki/LightGBM


proto_labs = list(lab_df['label'])

#Setting the objects to category 
cat_data = lab_df.drop('label', axis='columns')

for i in cat_data.select_dtypes(include='object'):
    cat_data[i] = cat_data[i].astype('category')

clf_kp = LGBMClassifier()

#cv_scores_kp = cross_val_score(clf_kp, cat_data, lab_df, scoring='f1_weighted')
#print(f'CV F1 score for K-Prototypes clusters is {np.mean(cv_scores_kp)}')

# So this method is taking a weak learner, a gradient boosted model, to see how well the data maps onto
# the cluster labels using the categories
#(https://en.wikipedia.org/wiki/Gradient_boosting#Gradient_tree_boosting)

In [ ]:
# https://shap-lrjball.readthedocs.io/en/docs_update/generated/shap.TreeExplainer.html
clf_kp.fit(cat_data, proto_labs)
explainer_kp = shap.TreeExplainer(clf_kp)
shap_values_kp = explainer_kp.shap_values(cat_data)


In [ ]:
shap.summary_plot(shap_values_kp, cat_data, color=plt.get_cmap("tab10"), show=False)
fig = plt.gcf()
fig.set_figheight(12)
fig.set_figwidth(14)
plt.legend(loc='lower right')
plt.savefig('shaps_622.png')

In [ ]:
##################################### Repeat analysis for SE Subset ###########################################

se_kproto_model, se_kproto_cluster, se_lab_df = kproto_trainer(data_frame=se_prepped, random_state=rando_n, k=3, num_init=3)
shap.summary_plot(shap_values_kp, cat_data, color=plt.get_cmap("tab10"), show=False)
fig = plt.gcf()
fig.set_figheight(12)
fig.set_figwidth(14)
plt.legend(loc='lower right')
plt.savefig('shaps_622.png')
# Print cluster centroids of the trained model.
print(se_kproto_model.cluster_centroids_)

# Print training statistics
print(se_kproto_model.cost_)
print(se_kproto_model.n_iter_)


#kproto_model, kproto_cluster, lab_df
se_lab_df

###########################################

from prince import FAMD

se_famd = FAMD(n_components=2, n_iter=3, check_input=True, engine='auto', random_state=rando_n)

se_famd = se_famd.fit(se_lab_df.drop('label', axis='columns'))
se_famd.fit(se_lab_df)
se_famd.transform(se_lab_df)

clx = se_famd.plot_row_coordinates(se_lab_df, figsize=(15,10), color_labels = se_lab_df['label'])

clx.get_figure().savefig('se_clusters_622.png')

print(pd.Series(se_kproto_cluster).value_counts())



In [ ]:
# Save Model

model_file = "SE_kproto_cluster_6_22.pkl"  

pickle.dump(se_kproto_model, open(model_file, 'wb'))



In [ ]:
#############################################

se_proto_labs = list(se_lab_df['label'])

#Setting the objects to category 
se_cat_data = se_lab_df.drop('label', axis='columns')


for i in se_cat_data.select_dtypes(include='object'):
    se_cat_data[i] = se_cat_data[i].astype('category')

se_clf_kp = LGBMClassifier()

# https://shap-lrjball.readthedocs.io/en/docs_update/generated/shap.TreeExplainer.html
se_clf_kp.fit(se_cat_data, se_proto_labs)
se_explainer_kp = shap.TreeExplainer(se_clf_kp)
se_shap_values_kp = se_explainer_kp.shap_values(se_cat_data)
#se_shap_values_kp


In [ ]:
shap.summary_plot(se_shap_values_kp, se_cat_data, color=plt.get_cmap("tab10"), show=False)
fig = plt.gcf()
fig.set_figheight(12)
fig.set_figwidth(14)
plt.legend(loc='lower right')
plt.savefig('se_shaps_622.png')